In [1]:
import requests
from datetime import datetime
import pandas as pd
df=pd.read_csv('df13.csv')
import json
def get_codeforces_rating_history(username):
    url = f"https://codeforces.com/api/user.rating?handle={username}"
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            data = response.json()
            if data["status"] == "OK":
                return data["result"]
            else:
                print(f"API Error: {data.get('comment', 'No specific error message provided')}")
        except json.JSONDecodeError:
            print("Invalid JSON response from the API")
    else:
        print(f"Failed to fetch rating history for {username}. Status code: {response.status_code}")

    return None

def days_to_cross_margin(username, target_lower, target_upper):
    rating_history = get_codeforces_rating_history(username)
    
    if rating_history is None:
        return None

    current_rating = None
    start_date = None
    end_date = None

    for entry in rating_history:
        rating = entry["newRating"]
        contest_time = entry["ratingUpdateTimeSeconds"]
        contest_date = datetime.fromtimestamp(contest_time)

        if rating <= target_lower:
            start_date = contest_date
        elif rating >= target_upper:
            end_date = contest_date
            break

    if start_date and end_date:
        days_taken = (end_date - start_date).days
        return days_taken
    else:
        print(f"{username} did not cross the rating margin within the specified range.")
        return None

def add_result_to_dataframe(df):
    df['Result'] = ""
    target_lower = 1200
    target_upper = 1400

    for index, row in df.iterrows():
        username = row['Handle']
        days_taken = days_to_cross_margin(username, target_lower, target_upper)
        
        if days_taken is not None:
            if 20 <= days_taken <= 40:
                df.at[index, 'Result'] = "Yes"
            else:
                df.at[index, 'Result'] = "No"
        else:
            df.at[index, 'Result'] = "N/A"

    return df

if __name__ == "__main__":
    
    df = add_result_to_dataframe(df)

    print(df)

Failed to fetch rating history for a_yong. Status code: 503
bqn did not cross the rating margin within the specified range.
Failed to fetch rating history for coderfrog506. Status code: 503
Failed to fetch rating history for npro07143. Status code: 503
tumu1t did not cross the rating margin within the specified range.
Failed to fetch rating history for xiaofanger. Status code: 503
Failed to fetch rating history for Not-Dola_. Status code: 503
Failed to fetch rating history for aniervs. Status code: 503
crude_human did not cross the rating margin within the specified range.
Failed to fetch rating history for e____r____. Status code: 503
Failed to fetch rating history for grdrdasari02. Status code: 503
Failed to fetch rating history for mcrash12. Status code: 503
Failed to fetch rating history for A.G.E. Status code: 503
Failed to fetch rating history for CEzi_dong_ji. Status code: 503
Failed to fetch rating history for Mehab. Status code: 503
Failed to fetch rating history for S4MI. Sta

In [2]:
df.head()

,Handle,Max Rating,Result
0,sachinjaiswal,1686,No
1,stone_hash,1571,No
2,ttirth1234518,1589,No
3,0x_3ea,1590,No
4,Abdalazim.ahmed,1597,No


In [3]:
df.to_csv('outdf13.csv', index=False)